# Metode Pengumpulan Data

> pada kernel ini saya melakukan scraping data ulasan restoran pada google maps Metode pengumpulan data yang digunakan adalah scraping data menggunakan library SerpApi. SerpApi adalah sebuah platform yang menyediakan API untuk melakukan scraping data pada berbagai platform seperti Google, Bing, dan lain sebagainya. Pada kernel ini, SerpApi digunakan untuk melakukan scraping data pada ulasan restoran yang ada di Makassar di Google Maps. Dengan menggunakan SerpApi, kita dapat dengan mudah mengambil data ulasan restoran secara otomatis tanpa harus melakukan pengambilan data secara manual satu persatu. Data yang berhasil diambil kemudian akan digunakan sebagai data latih dalam pembuatan model untuk melakukan klasifikasi sentimen ulasan restoran.

In [1]:
pip install google-search-results

  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32016 sha256=fc1f67a9d50d36f7899d8eec333cf8d37830343109194f74e2159cf09527245e
  Stored in directory: /Users/user/Library/Caches/pip/wheels/68/8e/73/744b7d9d7ac618849d93081a20e1c0deccd2aef90901c9f5a9
Successfully built google-search-results
Note: you may need to restart the kernel to use updated packages.


In [2]:
from serpapi import GoogleSearch
from urllib.parse import *

In [3]:
params = {
  "engine": "google_maps",
  "q": "restoran", # objek yang dicari
  "ll": "@-5.1229484,119.4022719,15z", #koordinat tempat sekitar
  "type": "search",
  "api_key": "1a6caebeae3bc6ec43936f11cbb04fe89cd9a32619a2d8f5ae30b8fc552e62d6"
}


search = GoogleSearch(params)

lIdx = 0 # indeks yang ditampilkan dari data
lSum = 7 # Batas jumlah data yang ditampilkan
local_results = []

while lIdx  <= lSum:
    results = search.get_dict()
    if "local_results" in results:
        for Result in results["local_results"]:
            lIdx += 1
            if lIdx <= lSum:
                if isinstance(Result, dict):
                    # seleksi data (jangan ambil data jika jumlah review < 200 )
                    reviews = Result.get("reviews", 0)
                    if isinstance(reviews, str):
                        reviews = int(reviews.replace(",", ""))
                    if reviews < 100:
                        continue
                    # Append data sesuai dengan batas jumlah yang sudah ditentukan
                    local_results.append({'Nama': Result["title"],
                                          'data_id': Result["data_id"],
                                          'total_reviews': reviews})
    #menghilangkan pagination pada web, sehingga data yang terambil bisa banyak
    if "next" in results.get("serpapi_pagination", {}):
        # Akan mengubah parameter dari 'GoogleSearch()' dengan isi parameter dari halaman selanjutnya
        search.params_dict.update(dict(parse_qsl(urlsplit(results.get("serpapi_pagination").get("next")).query)))
    else:
        break

print(local_results)


[{'Nama': 'Hong Kong Restaurant', 'data_id': '0x2dbf02a38a29c593:0xfc49c39b6e8ddd5d', 'total_reviews': 668}, {'Nama': 'RM. Pallu Kaloa 1', 'data_id': '0x2dbf029fa0a00d7f:0xe942ca9778839c9f', 'total_reviews': 1176}, {'Nama': 'Rumah Makan Patene', 'data_id': '0x2dbf02a4c9b715eb:0xee9b995f7f676d21', 'total_reviews': 335}, {'Nama': 'RICH TASTE', 'data_id': '0x2dbf02a1765eedad:0x96c967adaeea9e56', 'total_reviews': 417}]


In [4]:
for LR in local_results:
  print(LR)

{'Nama': 'Hong Kong Restaurant', 'data_id': '0x2dbf02a38a29c593:0xfc49c39b6e8ddd5d', 'total_reviews': 668}
{'Nama': 'RM. Pallu Kaloa 1', 'data_id': '0x2dbf029fa0a00d7f:0xe942ca9778839c9f', 'total_reviews': 1176}
{'Nama': 'Rumah Makan Patene', 'data_id': '0x2dbf02a4c9b715eb:0xee9b995f7f676d21', 'total_reviews': 335}
{'Nama': 'RICH TASTE', 'data_id': '0x2dbf02a1765eedad:0x96c967adaeea9e56', 'total_reviews': 417}


In [5]:
data = {'Restoran_name':[],
        'name':[],
        'rating':[],
        'review':[]}

for LR in local_results:
  
  params = {
    "engine": "google_maps_reviews",
    "data_id": "",
    "api_key": "08851987a422f702d52c8ac291e6a65769587f7b7fff05544fa7344c8d289dfe"
  }
  
  params["data_id"] = LR['data_id']

  search = GoogleSearch(params)

  lIdx = 0 # Angka indeks dari data
  lSum = 30 # Batas jumlah data yang ditampilkan

  while lIdx <= lSum:
    results = search.get_dict()

    for Result in results["reviews"]:
      lIdx += 1

      if lIdx <= lSum:
        # Append data sesuai dengan batas jumlah yang sudah ditentukan
        data['Restoran_name'].append(LR['Nama']);
        data['name'].append(Result["user"]["name"]);
        data['rating'].append(Result["rating"]);
        data['review'].append(Result["snippet"]);

    #menghilangkan pagination pada web, sehingga data yang terambil bisa banyak
    if "next" in results.get("serpapi_pagination", {}):
      # Akan mengubah parameter dari 'GoogleSearch()' dengan isi parameter dari halaman selanjutnya
      search.params_dict.update(dict(parse_qsl(urlsplit(results.get("serpapi_pagination").get("next")).query)))

    else:
      break
    
review_original = data['review']


In [ ]:
import os
import pandas as pd

# create directory if it does not exist
if not os.path.exists('File CSV'):
    os.makedirs('File CSV')

dfScrap = pd.DataFrame(data)
dfScrap.to_csv('File CSV/reviewrestoranmakassar.csv', index = False)